In [2]:
# this is a first test of the Rational Quadratic Spline
# see https://arxiv.org/pdf/1906.04032.pdf
# code taken from https://github.com/tensorflow/probability/blob/master/tensorflow_probability/python/bijectors/rational_quadratic_spline.py#L314
# IMPORTANT: needs nighly-build of tensorflow_probability (pip install tfp-nightly)


import numpy as np
from tensorflow.keras import layers, models
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
import matplotlib.pyplot as plt

# to do:
# plotting the splines of a given slice
# test trivial case (slope=1, width= 1/bins, height growing linearly)                                    works
# test inverse(forward(x)) = x                                                                           works 
# test forward_log_det_jacobian(x) = - test.inverse_log_det_jacobian(forward(x))                         works
# connect to NN
# create chain
# test inverse(forward(x))==x again
# get it to train in test cases


# diff to previous implementation of the piecewise: 
# rational quadratic spline istead of linear/quadratic function
# learn cdf instead of pdf: input values for heights are understood as additions in CDF, not positions in PDF. This might help since we dont have to normalize the pdf to 1 
# RQS takes objects of shape [something,nbins] but not [nbatch,ndims,nbins]: If we train one dimension based on D-1 others, this should be fine
# bijector is given to realNVP subclass, instead of bijector subclass

/home/claudius/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [67]:
def def_bin_positions(x):
    x = tf.reshape(x, [-1, nbins])
    return tf.math.softmax(x, axis=-1) * (2 - nbins * 1e-2) + 1e-2
def def_slopes(x):
    x = tf.reshape(x, [-1, nbins - 1])
    return tf.math.softplus(x) + 1e-2

In [84]:
# not really needed, only if we need the pdf (dist.prob should work without it) 
# or need a plotting routine for visualization
class RQS(tfb.RationalQuadraticSpline):
      
    def pdf(self, x):
        """Compute the pdf based on the forward derivative (Appendix A.2)."""
        d = self._compute_shared(x=x)
        relx = (x - d.x_k) / d.w_k
        relx = tf.where(d.out_of_bounds, tf.constant(.5, x.dtype), relx)
        ret =(
            (d.s_k**2) *
            (d.d_kp1 * relx**2 + 2 * d.s_k * relx * (1 - relx) +  # newln
                        d.d_k * (1 - relx)**2) /
            ((d.d_kp1 + d.d_k - 2 * d.s_k) * relx *
                        (1 - relx) + d.s_k)**2)
        return tf.where(d.out_of_bounds, tf.ones([], dtype=x.dtype), ret)

# work in progress (not done yet)
    def plot(self, batch, dim):
        pos = (batch*ndims) + dim
        num_pts=21
        wsum = np.cumsum(self.bin_widths[pos])
        x_pts = np.array([])
        for i in range(len(wsum)):
            if i == 0:
                x_temp = np.linspace(0.,wsum[i],num_pts)
            else:
                x_temp = np.linspace(wsum[i-1],wsum[i],num_pts)
            x_pts = np.append(x_pts,x_temp)
        

In [75]:
# testing random inputs
nbins = 10
ndims = 3
nbatch = 5

# output of NN:
wdths =10* (np.random.rand(nbatch, ndims * nbins) -0.5)
hgths = 10* (np.random.rand(nbatch, ndims * nbins)-0.5)
slopes = 10* (np.random.rand(nbatch, ndims * (nbins-1))-0.5)

# to be transformed:
x = np.random.rand(nbatch*ndims)

#wdths = np.cumsum(def_bin_positions(wdths).numpy(),axis=-1)
#hgths = np.cumsum(def_bin_positions(hgths).numpy(),axis=-1)

wdths = def_bin_positions(wdths).numpy()
hgths = def_bin_positions(hgths).numpy()
slopes = def_slopes(slopes).numpy()

In [59]:
# testing the trivial input (PDF = const.), so spline gives identity
nbins = 10
ndims = 3
nbatch = 5
wdths = 2./nbins * np.ones((nbatch,ndims,nbins))
hgths = 2./nbins * np.ones((nbatch,ndims,nbins))
slopes = np.ones((nbatch,ndims,nbins-1))
x = np.random.rand(nbatch,ndims)

In [76]:
print(wdths)
print("* * *")
print(hgths)
print("* * *")
print(slopes)

[[0.01062701 0.44698609 0.88807352 0.31895391 0.01262714 0.09540951
  0.01553157 0.01038709 0.10523334 0.09617082]
 [0.10737736 0.13707638 0.08294339 0.01154945 0.01500154 0.032172
  0.01093411 0.01365167 0.88501771 0.70427638]
 [0.04514664 0.01092627 0.01068276 0.01008854 0.61524203 0.0100493
  0.50411169 0.77071532 0.01172272 0.01131472]
 [0.07803577 0.89690438 0.05633405 0.01774789 0.10382021 0.30957109
  0.01119823 0.01149968 0.50115106 0.01373764]
 [0.69285221 0.03506699 0.01025391 0.03731003 0.6410812  0.01021585
  0.0112968  0.10328866 0.34635089 0.11228345]
 [0.01207218 0.01559577 1.61096655 0.04576444 0.02198236 0.01147645
  0.01480174 0.02082295 0.06493018 0.18158738]
 [0.02860643 0.78969818 0.16203459 0.86425763 0.01015273 0.01202997
  0.02598383 0.0366721  0.05743783 0.01312672]
 [0.14120922 1.00626352 0.03326484 0.01035465 0.28526682 0.01059831
  0.10052557 0.39078352 0.01141013 0.01032342]
 [0.06990034 0.01052934 0.01220096 0.25181935 0.01035256 0.24334946
  0.01085924 0.

In [85]:
# range_min defines start of integration region. End is given by range_min + Sum (widths)
# default is -1 to 1
test=RQS(wdths,hgths,slopes,range_min=0.)

In [86]:
#print(x)
#print(test.forward(x))
#print(test.inverse(test.forward(x)))
print(np.allclose(x,test.inverse(test.forward(x)).numpy()))
print(np.allclose(test.forward_log_det_jacobian(x,event_ndims=0), - test.inverse_log_det_jacobian(test.forward(x),event_ndims=0)))
print(np.allclose(tf.math.log(test.pdf(x)),test.forward_log_det_jacobian(x,event_ndims=0).numpy()))

True
True
True


In [ ]:
### Taken from github docstring:

In [94]:
nsplits = 3
class SplineParams(tf.Module):
    def __init__(self, nbins=32):
        self._nbins = nbins
        self._built = False
        self._bin_widths = None
        self._bin_heights = None
        self._knot_slopes = None
    def _bin_positions(self, x):
        x = tf.reshape(x, [-1, self._nbins])
        return tf.math.softmax(x, axis=-1) * (2 - self._nbins * 1e-2) + 1e-2
    def _slopes(self, x):
        x = tf.reshape(x, [-1, self._nbins - 1])
        return tf.math.softplus(x) + 1e-2
    def __call__(self, x, nunits):
        if not self._built:
            self._bin_widths = tf.keras.layers.Dense(
            nunits * self._nbins, activation=self._bin_positions, name='w')
            self._bin_heights = tf.keras.layers.Dense(
            nunits * self._nbins, activation=self._bin_positions, name='h')
            self._knot_slopes = tf.keras.layers.Dense(
            nunits * (self._nbins - 1), activation=self._slopes, name='s')
            self._built = True
        return tfb.RationalQuadraticSpline(
              bin_widths=self._bin_widths(x),
              bin_heights=self._bin_heights(x),
              knot_slopes=self._knot_slopes(x))
  
xs = np.random.rand(1, 15).astype(np.float32)  # Keras won't Dense(.)(vec).
splines = [SplineParams() for _ in range(nsplits)]
def spline_flow():
    stack = tfb.Identity()
    for i in range(nsplits):
        stack = tfb.RealNVP(5 * i, bijector_fn=splines[i])(stack)
    return stack
ys = spline_flow().forward(xs)
ys_inv = spline_flow().inverse(ys)  # ys_inv ~= xs

In [95]:
xs

array([[0.31561786, 0.02783527, 0.29327342, 0.17290546, 0.9759356 ,
        0.64925086, 0.02561379, 0.7627006 , 0.4975027 , 0.26890928,
        0.22438319, 0.565388  , 0.7774883 , 0.42874184, 0.7555837 ]],
      dtype=float32)

In [96]:
ys

<tf.Tensor: id=65542, shape=(1, 15), dtype=float32, numpy=
array([[0.3148033 , 0.02724843, 0.29501393, 0.17488578, 0.9741162 ,
        0.6441201 , 0.04591431, 0.7533772 , 0.5150754 , 0.28693557,
        0.1879066 , 0.535685  , 0.7786635 , 0.42972386, 0.7815804 ]],
      dtype=float32)>

In [97]:
ys_inv

<tf.Tensor: id=67804, shape=(1, 15), dtype=float32, numpy=
array([[0.31561786, 0.02783527, 0.29327342, 0.17290546, 0.9759356 ,
        0.64925086, 0.02561379, 0.7627006 , 0.49750274, 0.26890928,
        0.2243832 , 0.56538796, 0.7774883 , 0.42874187, 0.7555837 ]],
      dtype=float32)>